# Step 3: Define Answer Bounding Boxes
1. Convert the exam PDF into page images.
2. Auto-detect bounding boxes with AI.
3. Manually review and adjust each answer region.

**Features:**
- ✅ Comprehensive validation of input files and setup
- ✅ Robust OCR processing with retry logic and caching
- ✅ Progress tracking for multi-page processing
- ✅ Coordinate validation and scaling
- ✅ Robust error handling and recovery
- ✅ Detailed processing reports and validation summaries


The following command extracts cache for the sample to speed up and reduce costs for the demo.

In [1]:
from grading_utils import (
    setup_paths, create_directories, init_gemini_client, 
    validate_required_files, print_validation_summary
)
import logging
import time
import json
import os
import base64
from tqdm import tqdm
from agents.annotation_agent.agent import extract_annotations_with_ai
from typing import List
from google import genai
from google.genai import types
from pdf2image import convert_from_path
from PIL import Image
import copy

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

prefix = "VTC Test"
paths = setup_paths(prefix, "sample")
# Configuration - can be adjusted for testing
number_of_pages = 2  # Set to specific number for testing, or use len(pages) after conversion

# Validate required files exist
missing_files = validate_required_files(paths)
if missing_files:
    print("❌ Setup validation failed!")
    for file in missing_files:
        print(f"  Missing: {file}")
    raise FileNotFoundError("Please ensure all required files are present.")

pdf_file = paths["pdf_file"]



print("✅ Setup validation passed")

Bad pipe message: %s [b' 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Sa']
Bad pipe message: %s [b'ri/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/', b'ng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nAccept-Encoding: gzip, deflate, br, zstd\r\nA']
Bad pipe message: %s [b'ept-Language: en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7\r\nPriority: u=0, i\r\nReferer: https://studio.fireb', b'e.google.com/\r\nSec-Ch-Ua: "Google Chrome";v="143", "Chromium";v="143", "Not A(Brand";v="24"\r\nSec', b'h-Ua-Arch: "x86"\r\nSec-Ch-Ua-Bitness: "64"\r\nS', b'-Ch-Ua-Form-Factors: "Desktop"\r\nSec-Ch-Ua-Full-Version: "143.0.7499.170"\r\nSec-Ch-Ua-Full-Version-Lis', b' "Google Chrome";v="143.0.7499.170", "Chromium";v="143.0.7499.170", "Not A(Brand";v="24.0.0.0"\r\nSec-Ch-Ua-Mobile: ?']
Bad pipe message: %s [b'\nSec-Ch-Ua-Model: ""\r\nSec-Ch-Ua-Platform: "Wind', b's"\r\nSec-Ch-Ua-Platform-Version: "19.0.0"\r\nSec-Ch-Ua-Wow64: ?

✅ Setup validation passed


In [2]:
# Robust directory creation and PDF conversion
try:
    # Extract paths from setup
    file_name = paths["file_name"]
    base_path = paths["base_path"]
    base_path_images = paths["base_path_images"]
    base_path_annotations = paths["base_path_annotations"]

    # Create directories with error handling
    create_directories(paths)
    logger.info("✓ Created all necessary directories")

    # Convert PDF to images with progress tracking
    logger.info("Converting PDF to images...")
    start_time = time.time()
    
    pages = convert_from_path(pdf_file, fmt='jpeg')
    conversion_time = time.time() - start_time
    
    logger.info(f"✓ Converted PDF to {len(pages)} images in {conversion_time:.2f}s")
    
    # Save images with progress tracking
    for count, page in enumerate(tqdm(pages, desc="Saving images")):
        image_path = f'{base_path_images}{count}.jpg'
        page.save(image_path, 'JPEG')
    
    logger.info(f"✓ Saved {len(pages)} images to {base_path_images}")
    
except Exception as e:
    logger.error(f"Failed to convert PDF or create directories: {e}")
    raise

2026-01-09 07:28:46,825 - INFO - ✓ Created all necessary directories
2026-01-09 07:28:46,827 - INFO - Converting PDF to images...
2026-01-09 07:28:46,827 - INFO - Converting PDF to images...
2026-01-09 07:28:49,546 - INFO - ✓ Converted PDF to 8 images in 2.72s
Saving images: 100%|██████████| 8/8 [00:00<00:00, 17.86it/s]
2026-01-09 07:28:50,000 - INFO - ✓ Saved 8 images to ../marking_form/VTC Test/images/


In [3]:
# Robust utility functions with error handling
def update_json_file(annotations, path):
    """Update JSON file with error handling."""
    try:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, "w") as f:
            json.dump(annotations, f, indent=4)
        logger.info(f"✓ Updated annotations file: {path}")
    except Exception as e:
        logger.error(f"Failed to update JSON file {path}: {e}")
        raise

def image_to_data_url(filename):
    """Convert image to data URL with error handling."""
    try:
        ext = filename.split(".")[-1].lower()
        if ext == 'jpg':
            ext = 'jpeg'
        prefix = f"data:image/{ext};base64,"
        
        with open(filename, "rb") as f:
            img = f.read()
        return prefix + base64.b64encode(img).decode("utf-8")
    except Exception as e:
        logger.error(f"Failed to convert image to data URL {filename}: {e}")
        raise

print("✓ Utility functions defined")

✓ Utility functions defined


In [4]:
# The extraction prompt is now encapsulated within the annotation_agent
logger.info("🔍 Using agent-managed prompt for extraction")

print("🔍 Starting bounding box extraction...")
print(f"Processing {number_of_pages} pages with OCR")

aiAnnotation = {}
processing_stats = {
    'total_pages': number_of_pages,
    'successful_pages': 0,
    'failed_pages': 0,
    'total_boxes': 0,
    'processing_time': 0
}

start_time = time.time()

# Process each page with progress tracking
for i in tqdm(range(number_of_pages), desc="Processing pages"):
    image_path = base_path_images + f"{i}.jpg"
    
    print(f"\n{'='*60}")
    print(f"Processing page {i} ({image_path})")
    print(f"{'='*60}")
    
    try:
        # Validate image exists
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image file not found: {image_path}")
        
        # Use OCR with retry logic
        result = await extract_annotations_with_ai(image_path)
        
        # Convert Pydantic model to dict and extract boxes
        boxes_dict = [box.model_dump() for box in result.boxes]
        aiAnnotation[str(i)] = boxes_dict
        
        processing_stats['successful_pages'] += 1
        processing_stats['total_boxes'] += len(boxes_dict)
        
        print(f"✓ Page {i}: Found {len(boxes_dict)} bounding boxes")
        if boxes_dict:
            print(json.dumps(boxes_dict, indent=2))
        
        # Validate bounding boxes
        for box in boxes_dict:
            if box['width'] <= 0 or box['height'] <= 0:
                logger.warning(f"Invalid box dimensions on page {i}: {box}")
        if i != 0:
            filtered_boxes = [b for b in boxes_dict if b['label'] not in {'NAME', 'ID', 'CLASS'}]
            removed = len(boxes_dict) - len(filtered_boxes)
            if removed:
                print(f"Removed {removed} NAME/ID/CLASS boxes from page {i}")
            boxes_dict = filtered_boxes
            aiAnnotation[str(i)] = boxes_dict
            processing_stats['total_boxes'] -= removed
        if not boxes_dict:
            print("  (No bounding boxes detected)")
            logger.warning(f"No bounding boxes found on page {i}")
    
    except Exception as e:
        logger.error(f"Failed to process page {i}: {type(e).__name__}: {e}")
        aiAnnotation[str(i)] = []
        processing_stats['failed_pages'] += 1

processing_stats['processing_time'] = time.time() - start_time

print(f"\n{'='*60}")
print("✅ BOUNDING BOX EXTRACTION COMPLETED!")
print(f"{'='*60}")
print(f"📊 Processing Statistics:")
print(f"   Total pages: {processing_stats['total_pages']}")
print(f"   Successful: {processing_stats['successful_pages']}")
print(f"   Failed: {processing_stats['failed_pages']}")
print(f"   Total boxes found: {processing_stats['total_boxes']}")
print(f"   Processing time: {processing_stats['processing_time']:.2f}s")
print(f"   Average per page: {processing_stats['processing_time']/number_of_pages:.2f}s")
print(f"{'='*60}")

backup = copy.deepcopy(aiAnnotation)


2026-01-09 07:28:50,037 - INFO - 🔍 Using agent-managed prompt for extraction


🔍 Starting bounding box extraction...
Processing 2 pages with OCR


Processing pages:   0%|          | 0/2 [00:00<?, ?it/s]2026-01-09 07:28:50,044 - INFO - AI execution attempt 1/3 for annotation_extractor



Processing page 0 (../marking_form/VTC Test/images/0.jpg)


2026-01-09 07:28:50,351 - INFO - Sending out request, model: gemini-3-flash-preview, backend: GoogleLLMVariant.VERTEX_AI, stream: False
2026-01-09 07:28:50,353 - INFO - AFC is enabled with max remote calls: 10.
2026-01-09 07:29:08,277 - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/publishers/google/models/gemini-3-flash-preview:generateContent "HTTP/1.1 200 OK"
2026-01-09 07:29:08,283 - INFO - Response received from the model.
2026-01-09 07:29:08,286 - INFO - ✓ Successfully extracted 6 boxes via ADK output state!
Processing pages:  50%|█████     | 1/2 [00:18<00:18, 18.25s/it]2026-01-09 07:29:08,292 - INFO - AI execution attempt 1/3 for annotation_extractor
2026-01-09 07:29:08,470 - INFO - Sending out request, model: gemini-3-flash-preview, backend: GoogleLLMVariant.VERTEX_AI, stream: False
2026-01-09 07:29:08,473 - INFO - AFC is enabled with max remote calls: 10.


✓ Page 0: Found 6 bounding boxes
[
  {
    "x": 134,
    "y": 205,
    "width": 134,
    "height": 17,
    "label": "NAME"
  },
  {
    "x": 583,
    "y": 205,
    "width": 241,
    "height": 17,
    "label": "ID"
  },
  {
    "x": 134,
    "y": 233,
    "width": 84,
    "height": 17,
    "label": "CLASS"
  },
  {
    "x": 134,
    "y": 260,
    "width": 719,
    "height": 110,
    "label": "Q1"
  },
  {
    "x": 134,
    "y": 370,
    "width": 719,
    "height": 124,
    "label": "Q2"
  },
  {
    "x": 134,
    "y": 494,
    "width": 719,
    "height": 109,
    "label": "Q3"
  }
]

Processing page 1 (../marking_form/VTC Test/images/1.jpg)


2026-01-09 07:29:34,686 - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/publishers/google/models/gemini-3-flash-preview:generateContent "HTTP/1.1 200 OK"
2026-01-09 07:29:34,692 - INFO - Response received from the model.
2026-01-09 07:29:34,695 - INFO - ✓ Successfully extracted 5 boxes via ADK output state!
Processing pages: 100%|██████████| 2/2 [00:44<00:00, 22.33s/it]

✓ Page 1: Found 5 bounding boxes
[
  {
    "x": 135,
    "y": 178,
    "width": 132,
    "height": 20,
    "label": "NAME"
  },
  {
    "x": 584,
    "y": 178,
    "width": 240,
    "height": 20,
    "label": "ID"
  },
  {
    "x": 135,
    "y": 206,
    "width": 80,
    "height": 20,
    "label": "CLASS"
  },
  {
    "x": 135,
    "y": 234,
    "width": 719,
    "height": 111,
    "label": "Q4"
  },
  {
    "x": 135,
    "y": 345,
    "width": 719,
    "height": 124,
    "label": "Q5"
  }
]
Removed 3 NAME/ID/CLASS boxes from page 1

✅ BOUNDING BOX EXTRACTION COMPLETED!
📊 Processing Statistics:
   Total pages: 2
   Successful: 2
   Failed: 0
   Total boxes found: 8
   Processing time: 44.66s
   Average per page: 22.33s


In [5]:
# Robust coordinate scaling and validation
import copy
from PIL import Image

# Get image dimensions for scaling
sample_image_path = base_path_images + "0.jpg"
try:
    # Ensure backup exists (in case bounding box cell wasn't run or failed)
    if 'backup' not in locals():
        logger.warning("⚠️ Backup not found - loading AI annotations directly")
        ai_annotations_path = base_path_annotations + "ai_annotations.json"
        if os.path.exists(ai_annotations_path):
            with open(ai_annotations_path, "r") as f:
                backup = json.load(f)
            logger.info("✓ Loaded backup from saved AI annotations")
        else:
            logger.error("❌ No backup found and no saved annotations available")
            raise FileNotFoundError("AI annotations not found. Please run the bounding box extraction cell first.")
    
    with Image.open(sample_image_path) as img:
        width, height = img.size
    
    logger.info(f"✓ Image dimensions: {width}x{height}")
    print(f"Image dimensions: Width: {width}, Height: {height}")
    
    # Calculate scaling factors
    x_scale = width / 1000.0
    y_scale = height / 1000.0
    
    logger.info(f"Scaling factors: x={x_scale:.3f}, y={y_scale:.3f}")
    
    # Apply scaling with validation
    aiAnnotation = copy.deepcopy(backup)
    scaling_stats = {'scaled_boxes': 0, 'invalid_boxes': 0}
    
    for i in range(number_of_pages):
        for item in aiAnnotation[str(i)]:
            # Store original values for validation
            orig_x, orig_y = item['x'], item['y']
            orig_w, orig_h = item['width'], item['height']
            
            # Apply scaling
            item['x'] = int(round(item['x'] * x_scale))
            item['y'] = int(round(item['y'] * y_scale))
            item['width'] = int(round(item['width'] * x_scale))
            item['height'] = int(round(item['height'] * y_scale))
            
            # Validate scaled coordinates
            if (item['x'] < 0 or item['y'] < 0 or 
                item['x'] + item['width'] > width or 
                item['y'] + item['height'] > height):
                logger.warning(f"Scaled box out of bounds on page {i}: {item}")
                scaling_stats['invalid_boxes'] += 1
            else:
                scaling_stats['scaled_boxes'] += 1
    
    print(f"\n📐 Coordinate Scaling Results:")
    print(f"   Successfully scaled: {scaling_stats['scaled_boxes']} boxes")
    print(f"   Invalid after scaling: {scaling_stats['invalid_boxes']} boxes")
    
    # Save AI annotations
    ai_annotations_path = base_path_annotations + "ai_annotations.json"
    
    with open(ai_annotations_path, "w") as f:
        json.dump(aiAnnotation, f, indent=2)
    
    logger.info(f"✓ Saved AI annotations to: {ai_annotations_path}")
    print(f"✓ AI annotations saved to: {ai_annotations_path}")
    
except Exception as e:
    logger.error(f"Failed to process image dimensions or scaling: {e}")
    raise

2026-01-09 07:29:34,736 - INFO - ✓ Image dimensions: 1654x2338
2026-01-09 07:29:34,738 - INFO - Scaling factors: x=1.654, y=2.338
2026-01-09 07:29:34,741 - INFO - ✓ Saved AI annotations to: ../marking_form/VTC Test/annotations/ai_annotations.json


Image dimensions: Width: 1654, Height: 2338

📐 Coordinate Scaling Results:
   Successfully scaled: 8 boxes
   Invalid after scaling: 0 boxes
✓ AI annotations saved to: ../marking_form/VTC Test/annotations/ai_annotations.json


## Manual Annotation Review and Adjustment

Please ensure the following are clearly marked on each page before grading:
- **ID**: Student identification number
- **NAME**: Student name field
- **CLASS**: Student class/section

Use the interactive widget below to review and adjust the AI-generated bounding boxes.

In [6]:
# Robust interactive annotation widget with comprehensive features
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import glob

# Initialize widget state
page = 1
pageAndBoundingBoxes = {}

# Get all image files
files = sorted(glob.glob(base_path_images + "*.jpg"))
logger.info(f"Found {len(files)} image files for annotation")

# Create progress widget
w_progress = widgets.IntProgress(
    value=0, 
    max=len(files), 
    description="Progress",
    style={'description_width': 'initial'}
)

# File paths
annotations_path = base_path_annotations + "annotations.json"
ai_annotations_path = base_path_annotations + "ai_annotations.json"

# Load existing annotations with priority: manual > AI
annotations = {}

# Load AI annotations first (as base)
if os.path.exists(ai_annotations_path):
    try:
        with open(ai_annotations_path, "r") as f: 
            annotations = json.load(f)
        logger.info(f"✓ Loaded AI annotations for {len(annotations)} pages")
        print(f"✓ Loaded AI annotations for {len(annotations)} pages")
    except Exception as e:
        logger.error(f"Failed to load AI annotations: {e}")

# Then merge/override with manual annotations if they exist
if os.path.exists(annotations_path):
    try:
        with open(annotations_path, "r") as f: 
            manual_annotations = json.load(f)
            annotations.update(manual_annotations)  # Manual annotations take priority
        logger.info(f"✓ Merged manual annotations for {len(manual_annotations)} pages")
        print(f"✓ Merged manual annotations for {len(manual_annotations)} pages")
    except Exception as e:
        logger.error(f"Failed to load manual annotations: {e}")

print(f"Total pages with annotations: {list(annotations.keys())}")

# Create question input widget
question_widget = widgets.Text(
    value="", 
    placeholder="Enter question label (e.g., '1', '2', 'NAME', 'ID')", 
    description="Question:",
    style={'description_width': 'initial'}
)

# Create status widget
status_widget = widgets.HTML(
    value="<b>Status:</b> Ready to annotate",
    description=""
)

# Create bbox widget
w_bbox = BBoxWidget(
    image=image_to_data_url(files[0]) if files else None
)
w_bbox.attach(question_widget, name="label")

# Load initial bounding boxes
initial_page = str(w_progress.value)
if initial_page in annotations:
    w_bbox.bboxes = annotations[initial_page]
    status_widget.value = f"<b>Status:</b> Loaded {len(annotations[initial_page])} boxes for page {w_progress.value}"
else:
    w_bbox.bboxes = []
    status_widget.value = f"<b>Status:</b> No annotations found for page {w_progress.value}"

# Robust skip function
def on_skip():
    if w_progress.value + 1 >= len(files):
        status_widget.value = f"<b>Status:</b> Already at the last page ({len(files)-1})"
        logger.info(f"Already at the last page ({len(files)-1})")
        return
    
    w_progress.value += 1
    current_page = str(w_progress.value)
    
    try:
        # Load new image in the widget
        image_file = files[w_progress.value]
        w_bbox.image = image_to_data_url(image_file)
        
        # Load bounding boxes for current page
        if current_page in annotations:
            w_bbox.bboxes = annotations[current_page]
            status_widget.value = f"<b>Status:</b> Loaded {len(annotations[current_page])} boxes for page {w_progress.value}"
            logger.info(f"✓ Loaded {len(annotations[current_page])} bounding boxes for page {w_progress.value}")
        else:
            w_bbox.bboxes = []
            status_widget.value = f"<b>Status:</b> No annotations found for page {w_progress.value}"
            logger.warning(f"⚠️ No annotations found for page {w_progress.value}")
            
    except Exception as e:
        status_widget.value = f"<b>Status:</b> Error loading page {w_progress.value}: {e}"
        logger.error(f"Error loading page {w_progress.value}: {e}")

w_bbox.on_skip(on_skip)

# Robust submit function
def on_submit():
    try:
        current_page = str(w_progress.value)
        
        # Save annotations for current image
        annotations[current_page] = w_bbox.bboxes
        update_json_file(annotations, annotations_path)
        
        status_widget.value = f"<b>Status:</b> Saved {len(w_bbox.bboxes)} annotations for page {w_progress.value}"
        logger.info(f"✓ Saved {len(w_bbox.bboxes)} annotations for page {w_progress.value}")
        
        # Move to next page
        on_skip()
        
    except Exception as e:
        status_widget.value = f"<b>Status:</b> Error saving annotations: {e}"
        logger.error(f"Error saving annotations: {e}")

w_bbox.on_submit(on_submit)

# Output widget for bbox changes
w_out = widgets.Output()

def on_bbox_change(change):
    w_out.clear_output(wait=True)
    with w_out:
        current_boxes = change["new"]
        print(f"Page {w_progress.value}: {len(current_boxes)} bounding boxes")
        if current_boxes:
            print(json.dumps(current_boxes, indent=2))
        pageAndBoundingBoxes[w_progress.value] = current_boxes

w_bbox.observe(on_bbox_change, names=["bboxes"])

# Create comprehensive widget container
w_container = widgets.VBox([
    widgets.HTML("<h3>📝 Robust Interactive Annotation Tool</h3>"),
    status_widget,
    widgets.HBox([
        question_widget,
        widgets.HTML("<i>Tip: Use 'NAME', 'ID', 'CLASS' for student info fields</i>")
    ]),
    w_progress,
    w_bbox,
    widgets.HTML("<b>Current Annotations:</b>"),
    w_out,
    widgets.HTML("""
    <div style='margin-top: 10px; padding: 10px; background-color: #f0f0f0; border-radius: 5px;'>
    <b>Instructions:</b><br>
    • Draw bounding boxes around answer areas<br>
    • Label each box with question number or field name<br>
    • Use 'Submit' to save and move to next page<br>
    • Use 'Skip' to move without saving<br>
    • Ensure NAME, ID, and CLASS fields are marked
    </div>
    """)
])

print("\n🎯 Interactive annotation widget ready!")
print("Use the widget below to review and adjust bounding boxes.")

w_container

2026-01-09 07:29:35,111 - INFO - Found 8 image files for annotation
2026-01-09 07:29:35,115 - INFO - ✓ Loaded AI annotations for 2 pages
2026-01-09 07:29:35,135 - INFO - ✓ Merged manual annotations for 2 pages


✓ Loaded AI annotations for 2 pages
✓ Merged manual annotations for 2 pages
Total pages with annotations: ['0', '1']

🎯 Interactive annotation widget ready!
Use the widget below to review and adjust bounding boxes.


In [7]:
# Final summary and validation
def generate_annotation_summary():
    """Generate comprehensive annotation summary and validation report."""
    
    print(f"\n{'='*70}")
    print("🎉 STEP 3: ANNOTATION EXTRACTION COMPLETED")
    print(f"{'='*70}")
    
    # Load final annotations
    final_annotations = {}
    if os.path.exists(annotations_path):
        with open(annotations_path, "r") as f:
            final_annotations = json.load(f)
    
    # Generate statistics
    total_pages = len(final_annotations)
    total_boxes = sum(len(boxes) for boxes in final_annotations.values())
    
    # Analyze annotation types
    label_counts = {}
    required_fields = ['NAME', 'ID', 'CLASS']
    pages_with_required = {field: 0 for field in required_fields}
    
    for page, boxes in final_annotations.items():
        page_labels = set()
        for box in boxes:
            label = box.get('label', 'Unknown')
            label_counts[label] = label_counts.get(label, 0) + 1
            page_labels.add(label)
        
        # Check for required fields
        for field in required_fields:
            if field in page_labels:
                pages_with_required[field] += 1
    
    print(f"📊 Annotation Statistics:")
    print(f"   Total pages annotated: {total_pages}")
    print(f"   Total bounding boxes: {total_boxes}")
    print(f"   Average boxes per page: {total_boxes/total_pages:.1f}" if total_pages > 0 else "   No pages annotated")
    
    print(f"\n🏷️ Label Distribution:")
    for label, count in sorted(label_counts.items()):
        print(f"   {label}: {count} boxes")
    
    print(f"\n✅ Required Field Coverage:")
    all_required_present = True
    for field in required_fields:
        coverage = pages_with_required[field]
        status = "✓" if coverage > 0 else "❌"
        print(f"   {status} {field}: Found on {coverage}/{total_pages} pages")
        if coverage == 0:
            all_required_present = False
    
    print(f"\n📁 Generated Files:")
    print(f"   ✅ AI annotations: {ai_annotations_path}")
    print(f"   ✅ Final annotations: {annotations_path}")
    print(f"   ✅ Page images: {base_path_images} ({len(files)} files)")
    
    print(f"\n🎯 Next Steps:")
    if all_required_present:
        print(f"   ✅ All required fields present - ready for Step 4")
        print(f"   1. Proceed to Step 4: Scoring Preprocessing")
        print(f"   2. The annotations will be used for answer extraction")
    else:
        print(f"   ⚠️ Missing required fields - please review annotations")
        print(f"   1. Use the annotation widget to add missing NAME/ID/CLASS fields")
        print(f"   2. Ensure all pages have student identification fields")
        print(f"   3. Then proceed to Step 4")
    
    print(f"\n💡 Quality Assurance:")
    print(f"   • Robust OCR with retry logic and caching")
    print(f"   • Comprehensive coordinate validation and scaling")
    print(f"   • Interactive review and adjustment capability")
    print(f"   • Detailed processing statistics and error handling")
    
    print(f"\n{'='*70}")
    print(f"✅ Robust Step 3 completed successfully!")
    print("Ready for answer extraction and grading!")
    print(f"{'='*70}")

# Generate the summary
generate_annotation_summary()



🎉 STEP 3: ANNOTATION EXTRACTION COMPLETED
📊 Annotation Statistics:
   Total pages annotated: 2
   Total bounding boxes: 8
   Average boxes per page: 4.0

🏷️ Label Distribution:
   CLASS: 1 boxes
   ID: 1 boxes
   NAME: 1 boxes
   Q1: 1 boxes
   Q2: 1 boxes
   Q3: 1 boxes
   Q4: 1 boxes
   Q5: 1 boxes

✅ Required Field Coverage:
   ✓ NAME: Found on 1/2 pages
   ✓ ID: Found on 1/2 pages
   ✓ CLASS: Found on 1/2 pages

📁 Generated Files:
   ✅ AI annotations: ../marking_form/VTC Test/annotations/ai_annotations.json
   ✅ Final annotations: ../marking_form/VTC Test/annotations/annotations.json
   ✅ Page images: ../marking_form/VTC Test/images/ (8 files)

🎯 Next Steps:
   ✅ All required fields present - ready for Step 4
   1. Proceed to Step 4: Scoring Preprocessing
   2. The annotations will be used for answer extraction

💡 Quality Assurance:
   • Robust OCR with retry logic and caching
   • Comprehensive coordinate validation and scaling
   • Interactive review and adjustment capability
  